# I confirm that this is my own work, except where clearly indicated.

All Data Compiled on May 5th

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
input_data = pd.read_csv("train.csv")

In [3]:
prediction_test_data = pd.read_csv("test.csv")
prediction_test_data['Date'] =  pd.to_datetime(prediction_test_data['Date'])
forecast_ids = prediction_test_data["ForecastId"]
prediction_test_data.drop("ForecastId", axis=1, inplace=True)
prediction_test_data["Country_Region"] = prediction_test_data["Country_Region"] + "_C"
prediction_test_data.Province_State.fillna(prediction_test_data.Country_Region, inplace=True)
prediction_test_data.drop("Country_Region", axis=1, inplace=True)
prediction_test_data.columns = ['Location', 'Date']

In [4]:
input_data['Date'] =  pd.to_datetime(input_data['Date'])

In [5]:
input_data = input_data.drop("Id", axis=1)

In [6]:
new_data = input_data.copy()

In [7]:
new_data["Country_Region"] = new_data["Country_Region"] + "_C"

In [8]:
new_data.Province_State.fillna(new_data.Country_Region, inplace=True)

In [9]:
new_data.drop("Country_Region", axis=1, inplace=True)

In [10]:
new_data.columns = ['Location', 'Date', 'ConfirmedCases','Fatalities']

In [11]:
new_data

,Location,Date,ConfirmedCases,Fatalities
0,Afghanistan_C,2020-01-22,0.0,0.0
1,Afghanistan_C,2020-01-23,0.0,0.0
2,Afghanistan_C,2020-01-24,0.0,0.0
3,Afghanistan_C,2020-01-25,0.0,0.0
4,Afghanistan_C,2020-01-26,0.0,0.0
...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0
20576,Zimbabwe_C,2020-03-28,7.0,1.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0


### Lockdown Dates

In [12]:
lockdown_dates = pd.read_csv("countryLockdownDates.csv")

In [13]:
lockdown_dates = lockdown_dates.drop("Reference", axis=1)

In [14]:
lockdown_dates["Country/Region"] = lockdown_dates["Country/Region"] + "_C"

In [15]:
lockdown_dates.Province.fillna(lockdown_dates["Country/Region"], inplace=True)

In [16]:
lockdown_dates.drop("Country/Region", axis =1, inplace=True)

In [17]:
lockdown_dates

,Province,Date,Type
0,Afghanistan_C,24/03/2020,Full
1,Albania_C,08/03/2020,Full
2,Algeria_C,24/03/2020,Full
3,Andorra_C,16/03/2020,Full
4,Angola_C,24/03/2020,Full
...,...,...,...
302,Venezuela_C,16/03/2020,Full
303,Vietnam_C,19/03/2020,Full
304,West Bank and Gaza_C,05/03/2020,Full
305,Zambia_C,NaN,None


In [18]:
lockdown_map = {'None':0, 'Partial':1, 'Full':2}

lockdown_dates.Type = lockdown_dates.Type.apply(lambda x: lockdown_map.get(x))

In [19]:
lockdown_dates.Date.fillna(0, inplace=True)

In [20]:
lockdown_dates.columns = ['Location', 'Lockdown_Date', 'Lockdown_Type']

In [21]:
merged = pd.merge(new_data, lockdown_dates, on='Location', how="left")

In [22]:
merged["Lockdown_Date"] = pd.to_datetime(merged['Lockdown_Date'])

In [23]:
date = merged["Date"]
lock_down_date = merged["Lockdown_Date"]
l_type = merged["Lockdown_Type"]

In [24]:
lockdown_status = []
for idx, value in enumerate(lock_down_date):
    if pd.isnull(value):
        lockdown_status.append(0)
    elif date[idx] < value:
        lockdown_status.append(0)
    else:
        lockdown_status.append(l_type[idx])

In [25]:
merged["Lockdown_Status"] = lockdown_status

In [26]:
print(merged[merged["Location"] == "France_C"].head(60))

      Location       Date  ConfirmedCases  Fatalities Lockdown_Date  \
8190  France_C 2020-01-22             0.0         0.0    2020-03-16   
8191  France_C 2020-01-23             0.0         0.0    2020-03-16   
8192  France_C 2020-01-24             2.0         0.0    2020-03-16   
8193  France_C 2020-01-25             3.0         0.0    2020-03-16   
8194  France_C 2020-01-26             3.0         0.0    2020-03-16   
8195  France_C 2020-01-27             3.0         0.0    2020-03-16   
8196  France_C 2020-01-28             4.0         0.0    2020-03-16   
8197  France_C 2020-01-29             5.0         0.0    2020-03-16   
8198  France_C 2020-01-30             5.0         0.0    2020-03-16   
8199  France_C 2020-01-31             5.0         0.0    2020-03-16   
8200  France_C 2020-02-01             6.0         0.0    2020-03-16   
8201  France_C 2020-02-02             6.0         0.0    2020-03-16   
8202  France_C 2020-02-03             6.0         0.0    2020-03-16   
8203  

In [27]:
merged.drop(["Lockdown_Date", "Lockdown_Type"], axis=1, inplace=True)

In [28]:
merged

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0
...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0


### Testing Data

Testing data taken from https://ourworldindata.org/grapher/full-list-total-tests-for-covid-19

In [29]:
testing = pd.read_csv("full-list-total-tests-for-covid-19.csv")

In [30]:
len(testing.Entity.unique())

88

In [31]:
testing["Date"] = pd.to_datetime(testing['Date'])

In [32]:
testing

,Entity,Code,Date,Total tests
0,Argentina,ARG,2020-04-08,13330
1,Argentina,ARG,2020-04-09,14850
2,Argentina,ARG,2020-04-10,16379
3,Argentina,ARG,2020-04-11,18027
4,Argentina,ARG,2020-04-13,19758
...,...,...,...,...
4088,Zimbabwe,ZWE,2020-04-29,7642
4089,Zimbabwe,ZWE,2020-04-30,8314
4090,Zimbabwe,ZWE,2020-05-01,9291
4091,Zimbabwe,ZWE,2020-05-02,10727


In [33]:
list_of_countries = input_data["Country_Region"].unique()

In [34]:
testing.columns = ['Location', 'Code', 'Date', 'Total tests']

In [35]:
testing["Location"] = [x + "_C" if x in list_of_countries else  x for x in testing["Location"]]

In [36]:
second_merged = pd.merge(merged, testing, how="left", on=["Location", "Date"])

In [37]:
second_merged.drop("Code", axis=1, inplace=True)

In [38]:
second_merged[~pd.isna(second_merged["Total tests"])]

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests
1154,Austria_C,2020-02-25,2.0,0.0,0.0,218.0
1155,Austria_C,2020-02-26,2.0,0.0,0.0,321.0
1156,Austria_C,2020-02-27,3.0,0.0,0.0,447.0
1157,Austria_C,2020-02-28,3.0,0.0,0.0,763.0
1158,Austria_C,2020-02-29,9.0,0.0,0.0,1649.0
...,...,...,...,...,...,...
20574,Zimbabwe_C,2020-03-26,3.0,1.0,0.0,165.0
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,188.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,194.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,241.0


In [39]:
second_merged["Total tests"].fillna(-1, inplace=True)

In [40]:
second_merged

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0,-1.0
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0,-1.0
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0,-1.0
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0,-1.0
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,188.0
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0,-1.0
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,194.0
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,241.0


### Country Info

In [41]:
cinfo = pd.read_csv("covid19countryinfo.csv")

In [42]:
cinfo["country"] = cinfo["country"] + "_C"

In [43]:
cinfo.region.fillna(cinfo.country, inplace=True)

In [44]:
cinfo.drop("country", axis=1, inplace=True)

In [45]:
cinfo.columns = ['Location' if x=='region' else x for x in cinfo.columns]

In [46]:
list_to_remove = ["alpha3code", "alpha2code", 'quarantine', 'schools', 'publicplace', 'gatheringlimit',
       'gathering', 'nonessential', 'firstcase','totalcases','tests', 'testpop',
       'active30', 'active31', 'active1', 'active2', 'active3', 'newcases30',
       'newcases31', 'newcases1', 'newcases2', 'newcases3', 'deaths',
       'newdeaths30', 'newdeaths31', 'newdeaths1', 'newdeaths2', 'newdeaths3',
       'recovered', 'critical30', 'critical31', 'critical1', 'critical2',
       'critical3', 'casediv1m', 'deathdiv1m']

In [47]:
cinfo.drop(list_to_remove, axis=1, inplace=True)

In [48]:
cinfo.columns

Index(['Location', 'pop', 'density', 'medianage', 'urbanpop', 'hospibed',
       'smokers', 'sex0', 'sex14', 'sex25', 'sex54', 'sex64', 'sex65plus',
       'sexratio', 'lung', 'femalelung', 'malelung', 'gdp2019', 'healthexp',
       'healthperpop', 'fertility', 'avgtemp', 'avghumidity'],
      dtype='object')

In [49]:
cinfo.fillna(-1, inplace=True)

In [50]:
cinfo

,Location,pop,density,medianage,urbanpop,hospibed,smokers,sex0,sex14,sex25,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan_C,"38,928,346",60.0,18.0,25.0,0.5,-1.0,1.05,1.03,1.03,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
1,Albania_C,"2,877,797",105.0,36.0,63.0,2.9,29.4,1.08,1.11,1.09,...,0.98,11.67,7.02,17.04,"15,418",774,268.955733,1.51,13.950000,69.900000
2,Algeria_C,"43,851,044",18.0,29.0,73.0,1.9,-1.0,1.05,1.05,1.05,...,1.03,8.77,5.03,12.81,"172,781","1,031",23.511413,2.70,19.566667,61.716667
3,Andorra_C,"77,265",164.0,45.0,88.0,2.5,32.5,1.07,1.05,1.08,...,1.06,-1.00,-1.00,-1.00,"91,527","5,949",76994.758300,1.40,7.383333,75.233333
4,Antigua and Barbuda_C,"97,929",223.0,34.0,26.0,3.8,-1.0,1.05,1.03,0.99,...,0.90,11.76,7.67,18.78,"1,688","1,105",11283.685120,2.00,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Washington,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
245,Washington DC,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
246,West Virginia,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000
247,Wisconsin,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.00,-1.00,-1.00,...,-1.00,-1.00,-1.00,-1.00,-1,-1,-1.000000,-1.00,-1.000000,-1.000000


In [51]:
merging = pd.merge(second_merged, cinfo, on='Location', how="left")

In [52]:
merging

,Location,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests,pop,density,medianage,urbanpop,...,sexratio,lung,femalelung,malelung,gdp2019,healthexp,healthperpop,fertility,avgtemp,avghumidity
0,Afghanistan_C,2020-01-22,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
1,Afghanistan_C,2020-01-23,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
2,Afghanistan_C,2020-01-24,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
3,Afghanistan_C,2020-01-25,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
4,Afghanistan_C,2020-01-26,0.0,0.0,0.0,-1.0,"38,928,346",60.0,18.0,25.0,...,1.03,37.62,36.31,39.33,"18,734",184,4.726633,5.12,6.216667,56.933333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,Zimbabwe_C,2020-03-27,5.0,1.0,2.0,188.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20576,Zimbabwe_C,2020-03-28,7.0,1.0,2.0,-1.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20577,Zimbabwe_C,2020-03-29,7.0,1.0,2.0,194.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000
20578,Zimbabwe_C,2020-03-30,7.0,1.0,2.0,241.0,"14,862,924",38.0,18.7,38.4,...,1.01,22.53,19.31,26.47,"12,818",182,12.245235,3.63,-1.000000,-1.000000


In [53]:
merging.fillna(-1, inplace=True)

In [54]:
merging.isnull().values.any()

False

In [55]:
merging.dtypes

Location                   object
Date               datetime64[ns]
ConfirmedCases            float64
Fatalities                float64
Lockdown_Status           float64
Total tests               float64
pop                        object
density                   float64
medianage                 float64
urbanpop                  float64
hospibed                  float64
smokers                   float64
sex0                      float64
sex14                     float64
sex25                     float64
sex54                     float64
sex64                     float64
sex65plus                 float64
sexratio                  float64
lung                      float64
femalelung                float64
malelung                  float64
gdp2019                    object
healthexp                  object
healthperpop              float64
fertility                 float64
avgtemp                   float64
avghumidity               float64
dtype: object

In [56]:
convert_list = ["pop", "gdp2019", "healthexp"]
for i in convert_list:
    merging[i] = [str(x).replace(",", "") for x in merging[i]] 

merging[convert_list] = merging[convert_list].astype('float64')

In [57]:
merging.dtypes

Location                   object
Date               datetime64[ns]
ConfirmedCases            float64
Fatalities                float64
Lockdown_Status           float64
Total tests               float64
pop                       float64
density                   float64
medianage                 float64
urbanpop                  float64
hospibed                  float64
smokers                   float64
sex0                      float64
sex14                     float64
sex25                     float64
sex54                     float64
sex64                     float64
sex65plus                 float64
sexratio                  float64
lung                      float64
femalelung                float64
malelung                  float64
gdp2019                   float64
healthexp                 float64
healthperpop              float64
fertility                 float64
avgtemp                   float64
avghumidity               float64
dtype: object

In [58]:
merging['Date']=merging['Date'].map(dt.datetime.toordinal)

In [59]:
data = pd.get_dummies(merging, columns=["Location"])

In [60]:
data

,Date,ConfirmedCases,Fatalities,Lockdown_Status,Total tests,pop,density,medianage,urbanpop,hospibed,...,Location_Washington,Location_West Virginia,Location_Western Australia,Location_Wisconsin,Location_Wyoming,Location_Xinjiang,Location_Yunnan,Location_Zambia_C,Location_Zhejiang,Location_Zimbabwe_C
0,737446,0.0,0.0,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,...,0,0,0,0,0,0,0,0,0,0
1,737447,0.0,0.0,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,...,0,0,0,0,0,0,0,0,0,0
2,737448,0.0,0.0,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,...,0,0,0,0,0,0,0,0,0,0
3,737449,0.0,0.0,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,...,0,0,0,0,0,0,0,0,0,0
4,737450,0.0,0.0,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20575,737511,5.0,1.0,2.0,188.0,14862924.0,38.0,18.7,38.4,1.7,...,0,0,0,0,0,0,0,0,0,1
20576,737512,7.0,1.0,2.0,-1.0,14862924.0,38.0,18.7,38.4,1.7,...,0,0,0,0,0,0,0,0,0,1
20577,737513,7.0,1.0,2.0,194.0,14862924.0,38.0,18.7,38.4,1.7,...,0,0,0,0,0,0,0,0,0,1
20578,737514,7.0,1.0,2.0,241.0,14862924.0,38.0,18.7,38.4,1.7,...,0,0,0,0,0,0,0,0,0,1


In [61]:
date = dt.datetime.toordinal(pd.to_datetime('27/03/2020'))

In [62]:
test_data = data[data["Date"] >= date]
training_data = data[data["Date"] < date]

In [63]:
Y_Train = training_data[["ConfirmedCases", "Fatalities"]]
X_Train = training_data.drop(["ConfirmedCases","Fatalities"], axis=1)

Y_Test = test_data[["ConfirmedCases", "Fatalities"]]
X_Test = test_data.drop(["ConfirmedCases","Fatalities"], axis=1)


In [64]:
X_Train

,Date,Lockdown_Status,Total tests,pop,density,medianage,urbanpop,hospibed,smokers,sex0,...,Location_Washington,Location_West Virginia,Location_Western Australia,Location_Wisconsin,Location_Wyoming,Location_Xinjiang,Location_Yunnan,Location_Zambia_C,Location_Zhejiang,Location_Zimbabwe_C
0,737446,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,-1.00,1.05,...,0,0,0,0,0,0,0,0,0,0
1,737447,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,-1.00,1.05,...,0,0,0,0,0,0,0,0,0,0
2,737448,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,-1.00,1.05,...,0,0,0,0,0,0,0,0,0,0
3,737449,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,-1.00,1.05,...,0,0,0,0,0,0,0,0,0,0
4,737450,0.0,-1.0,38928346.0,60.0,18.0,25.0,0.5,-1.00,1.05,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20570,737506,0.0,-1.0,14862924.0,38.0,18.7,38.4,1.7,16.65,1.03,...,0,0,0,0,0,0,0,0,0,1
20571,737507,0.0,15.0,14862924.0,38.0,18.7,38.4,1.7,16.65,1.03,...,0,0,0,0,0,0,0,0,0,1
20572,737508,0.0,-1.0,14862924.0,38.0,18.7,38.4,1.7,16.65,1.03,...,0,0,0,0,0,0,0,0,0,1
20573,737509,0.0,100.0,14862924.0,38.0,18.7,38.4,1.7,16.65,1.03,...,0,0,0,0,0,0,0,0,0,1


In [65]:
def log_y(Y):
    y_new = Y.copy()
    y_new["ConfirmedCases"] = y_new["ConfirmedCases"].apply(lambda x: math.log(x) if x != 0 else 
                                        math.log(x+0.001))
    y_new["Fatalities"] = y_new["Fatalities"].apply(lambda x: math.log(x) if x != 0 else 
                                        math.log(x+0.1))
    return y_new

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
forest = RandomForestRegressor()

scores = cross_val_score(forest, X_Train, Y_Train, cv=5, scoring='neg_mean_squared_log_error')
print(scores)
print(np.mean(scores))

In [66]:
from sklearn.metrics import mean_squared_log_error

In [ ]:
forest.fit(X_Train, Y_Train)
predicted = forest.predict(X_Test)
mean_squared_log_error(Y_Test,predicted)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()
scores = cross_val_score(knn, X_Train, Y_Train, cv=5, scoring='neg_mean_squared_log_error')
print(scores)
print(np.mean(scores))

In [ ]:
knn.fit(X_Train, Y_Train)
predicted = knn.predict(X_Test)
mean_squared_log_error(Y_Test, predicted)

In [ ]:
predicted

## NN 

In [67]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.random.set_seed(42)

In [68]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import optimizers

In [69]:
def get_y(y):
    return {"confirmed": y["ConfirmedCases"], "fatalities":y["Fatalities"]}

In [79]:

dropout_threshold = 0.0

input_layer= Input(shape=X_Train.shape[1:],)
d0 = keras.layers.Dropout(dropout_threshold)(input_layer)
n0 = keras.layers.BatchNormalization()(d0)
h0 = Dense(500, activation='relu')(n0)

d1 = keras.layers.Dropout(dropout_threshold)(h0)
n1 = keras.layers.BatchNormalization()(d1)
h1 = Dense(500, activation='relu')(n1)

d2 = keras.layers.Dropout(dropout_threshold)(h1)
n2 = keras.layers.BatchNormalization()(d2)
h2 = Dense(500, activation='relu')(n2)

d3 = keras.layers.Dropout(dropout_threshold)(h2)
n3 = keras.layers.BatchNormalization()(d3)

out1 = Dense(1,  activation='exponential', name="confirmed")(n3)
out2 = Dense(1,  activation='exponential', name="fatalities")(n3)


model = Model(inputs=input_layer, outputs=[out1,out2])

In [80]:
from tensorflow.keras import optimizers
model.compile(loss="mean_squared_logarithmic_error", optimizer="sgd")
early_stopping_cb_cc = keras.callbacks.EarlyStopping(monitor="confirmed_loss",
                                                     patience=5, 
                                                      restore_best_weights=True)
early_stopping_cb_ft = keras.callbacks.EarlyStopping(monitor="fatalities_loss",
                                                     patience=5, 
                                                      restore_best_weights=True)
history = model.fit(X_Train, get_y(Y_Train), 
          epochs=50, 
          callbacks=[early_stopping_cb_cc, early_stopping_cb_ft],
          verbose=1)

Train on 19110 samples
Epoch 1/50
19110/19110 [==============================] - 4s 194us/sample - loss: 1.4584 - confirmed_loss: 1.1673 - fatalities_loss: 0.2936
Epoch 2/50
19110/19110 [==============================] - 3s 144us/sample - loss: 0.7998 - confirmed_loss: 0.6640 - fatalities_loss: 0.1400
Epoch 3/50
17216/19110 [==========================>...] - ETA: 0s - loss: 0.6518 - confirmed_loss: 0.5375 - fatalities_loss: 0.1143

KeyboardInterrupt: 

In [ ]:
x = range(1,len(history.history["loss"])+1)

plt.plot(x, history.history["confirmed_loss"], label = "Confirmed Loss")
plt.plot(x, history.history["fatalities_loss"], label = "Fatalities Loss")
plt.plot(x, np.average([history.history["fatalities_loss"], history.history["confirmed_loss"]], axis=0), label = "Average")

plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.title('Loss')
plt.legend()
plt.show()

In [ ]:
# Training Loss Average
np.average([history.history["confirmed_loss"][-1], history.history["fatalities_loss"][-1]])

In [ ]:
model.evaluate(X_Test, get_y(Y_Test))

In [ ]:
predicted = model.predict(X_Test)

In [70]:
import math
fst = [math.ceil(val) for sublist in predicted[0] for val in sublist]
snd = [math.ceil(val) for sublist in predicted[1] for val in sublist]
mean_squared_log_error(Y_Test, list(zip(fst, snd)))

NameError: name 'predicted' is not defined

In [75]:
def make_model(train, 
               dropout_threshold,
               neurons, 
               activation):

    dropout_threshold = 0.0

    input_layer_t = Input(shape=train.shape[1:],)
    d0_t = keras.layers.Dropout(dropout_threshold)(input_layer_t)
    n0_t = keras.layers.BatchNormalization()(d0_t)
    h0_t = Dense(500, activation='relu')(n0_t)

    d1_t = keras.layers.Dropout(dropout_threshold)(h0_t)
    n1_t = keras.layers.BatchNormalization()(d1_t)
    h1_t = Dense(500, activation='relu')(n1_t)

    d2_t = keras.layers.Dropout(dropout_threshold)(h1_t)
    n2_t = keras.layers.BatchNormalization()(d2_t)
    h2_t = Dense(500, activation='relu')(n2_t)

    d3_t = keras.layers.Dropout(dropout_threshold)(h2_t)
    n3_t = keras.layers.BatchNormalization()(d3_t)

    out1_t = Dense(1,  activation='exponential', name="confirmed")(n3_t)
    out2_t = Dense(1,  activation='exponential', name="fatalities")(n3_t)
    
    new_model = Model(inputs=input_layer_t, outputs=[out1_t,out2_t])

    return new_model

In [76]:
def compile_and_run(n_model, x_train, y_train, lr, verbose, epochs):
    opt = optimizers.SGD(lr=lr)
    n_model.compile(loss="mean_squared_logarithmic_error", optimizer=opt)
    early_stopping_cb_cc = keras.callbacks.EarlyStopping(monitor="confirmed_loss",
                                                         patience=5, 
                                                          restore_best_weights=True)
    early_stopping_cb_ft = keras.callbacks.EarlyStopping(monitor="fatalities_loss",
                                                         patience=5, 
                                                          restore_best_weights=True)
    history = n_model.fit(X_Train, get_y(Y_Train), 
              epochs=epochs, 
              callbacks=[early_stopping_cb_cc, early_stopping_cb_ft],
              verbose=verbose)
    return history

In [77]:
params = {
    "dropout":[0,0.2,0.5],
    "neurons":[500, 1000],
    "lr":[1, 0.1, 0.01],
    "activation":["exponential", "relu"]
}

In [78]:
combination = []
for dropout in params["dropout"]:
    for neurons in params["neurons"]:
        for learning_rate in params["lr"]:
            for activation in params["activation"]:
                model = make_model(X_Train, dropout, neurons, activation)
                history = compile_and_run(model,
                                                X_Train, 
                                                Y_Train, 
                                                learning_rate, 
                                                verbose=1, 
                                                epochs=50)

                prediction = model.predict(X_Test)
                training_loss = np.average([history.history["confirmed_loss"][-1], 
                            history.history["fatalities_loss"][-1]])
                c_loss = [math.ceil(val) for sublist in predicted[0] for val in sublist]
                f_loss = [math.ceil(val) for sublist in predicted[1] for val in sublist]
                test_loss = mean_squared_log_error(Y_Test, list(zip(c_loss, f_loss)))

                combination.append(({
                                        "dropout":dropout,
                                        "neurons":neurons,
                                        "lr":learning_rate,
                                    },
                                    training_loss,
                                    test_loss,
                                    model, 
                                    history))

Train on 19110 samples
Epoch 1/50
19110/19110 [==============================] - 3s 169us/sample - loss: nan - confirmed_loss: nan - fatalities_loss: nan
Epoch 2/50
19110/19110 [==============================] - 3s 134us/sample - loss: nan - confirmed_loss: nan - fatalities_loss: nan
Epoch 3/50
19110/19110 [==============================] - 3s 147us/sample - loss: nan - confirmed_loss: nan - fatalities_loss: nan
Epoch 4/50
19110/19110 [==============================] - 3s 138us/sample - loss: nan - confirmed_loss: nan - fatalities_loss: nan
Epoch 5/50
19008/19110 [============================>.] - ETA: 0s - loss: nan - confirmed_loss: nan - fatalities_loss: nan

TypeError: object of type 'NoneType' has no len()